# Detection of Large-Language Model (LLM) Generated Text

# Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns


2024-07-10 11:07:14.053724: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-10 11:07:14.053861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-10 11:07:14.199699: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Загружаем и предобрабртываем данные

In [2]:
data1=pd.read_csv('/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv')
data2=pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')

In [3]:
train1_selected = data1[['text', 'label']].copy()

# Выбор необходимых столбцов из датасета train_1 и переименование 'generated' для соответствия столбцу 'label'
train2_selected = data2[['text', 'generated']].rename(columns={'generated': 'label'}).copy()

# Объединение выбранных столбцов из обоих датасетов
data = pd.concat([train1_selected, train2_selected], ignore_index=True)

# Отображение объединенных данных
print(data)


                                                    text  label
0      Phones\n\nModern humans today are always on th...      0
1      This essay will explain if drivers should or s...      0
2      Driving while the use of cellular devices\n\nT...      0
3      Phones & Driving\n\nDrivers should not be able...      0
4      Cell Phone Operation While Driving\n\nThe abil...      0
...                                                  ...    ...
46241  There has been a fuss about the Elector Colleg...      0
46242  Limiting car usage has many advantages. Such a...      0
46243  There's a new trend that has been developing f...      0
46244  As we all know cars are a big part of our soci...      0
46245  Cars have been around since the 1800's and hav...      0

[46246 rows x 2 columns]


In [4]:
# Удаляем дупликаты и нулевые значения
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)

# Анализ данных в датасетах

In [ ]:
data1_counts = data1['label'].value_counts().reset_index()
data2_counts = data2['generated'].value_counts().reset_index()
index = np.arange(2)

v1 = [data1_counts['count'][0], data1_counts['count'][1]]
v2 = [data2_counts['count'][0], data2_counts['count'][1]]

bw = 0.3
plt.title('Распределение текстов по методу написания')
bars1 = plt.bar(index, v1, bw, color='#FF9999', label='Дополнительные данные')
bars2 = plt.bar(index+bw, v2, bw, color='#FFCC99', label='Данные с соревнования')
plt.xticks(index + 0.5 * bw, ['Сгенерировано ИИ', 'Написано человеком'])
plt.legend()  # Добавление легенды

# Добавление значений на столбцы
for bar in bars1:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2.0, height, '%d' % int(height), ha='center', va='bottom')

for bar in bars2:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2.0, height, '%d' % int(height), ha='center', va='bottom')

plt.show()

# Векторизуем текст

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Инициализация TF-IDF векторизатора
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Вы можете настроить max_features по мере необходимости

# Токенизация и создание TF-IDF векторов для столбца 'text' объединенного набора данных
tfidf_vectors = tfidf_vectorizer.fit_transform(data['text'])

# Преобразование TF-IDF векторов в DataFrame для удобного анализа
tfidf_df = pd.DataFrame(tfidf_vectors.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Отображение DataFrame с TF-IDF
print(tfidf_df.head())

    00       000   10  100   11  114  118   12   13   14  ...      your  \
0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000   
1  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.193019   
2  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000   
3  0.0  0.060816  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000   
4  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000   

   yours  yourself  youth  youtube  zip  zone  zones  zoos  zygomatic  
0    0.0       0.0    0.0      0.0  0.0   0.0    0.0   0.0        0.0  
1    0.0       0.0    0.0      0.0  0.0   0.0    0.0   0.0        0.0  
2    0.0       0.0    0.0      0.0  0.0   0.0    0.0   0.0        0.0  
3    0.0       0.0    0.0      0.0  0.0   0.0    0.0   0.0        0.0  
4    0.0       0.0    0.0      0.0  0.0   0.0    0.0   0.0        0.0  

[5 rows x 5000 columns]


# Делим датасет на тренировочные и тестовые данные

In [6]:
X = tfidf_df.values 
y = data['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Кодируем

In [7]:
# Кодирование целевых меток (0 и 1) с использованием LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Архитектура нейросети

In [8]:
# Создаем модель для нейронной сети
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Обучение

In [9]:
history = model.fit(X_train, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_test, y_test_encoded), verbose=1)

Epoch 1/10
1122/1122 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9384 - loss: 0.1343 - val_accuracy: 0.9949 - val_loss: 0.0166
Epoch 2/10
1122/1122 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9984 - loss: 0.0060 - val_accuracy: 0.9942 - val_loss: 0.0198
Epoch 3/10
1122/1122 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9990 - loss: 0.0032 - val_accuracy: 0.9957 - val_loss: 0.0198
Epoch 4/10
1122/1122 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9995 - loss: 0.0012 - val_accuracy: 0.9950 - val_loss: 0.0281
Epoch 5/10
1122/1122 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9998 - loss: 8.5950e-04 - val_accuracy: 0.9947 - val_loss: 0.0275
Epoch 6/10
1122/1122 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9995 - loss: 0.0016 - val_accuracy: 0.9947 - val_loss: 0.0263
Epoch 7/10
1122/1122 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9995 - loss: 0.0015 - val_accuracy: 0.9938 - val_loss: 0.0306
Epoch 8/10
1122/1122 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9997 - loss: 5.41

In [10]:
# Точность на тестовых данных
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded)
print(f'Test Accuracy: {test_accuracy:.4f}')

281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9951 - loss: 0.0444
Test Accuracy: 0.9951


# Сохраняем модель

In [11]:
import pickle

with open('tfidf_tokenizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

model.save('text_classification_model.h5')


# Usage Code

In [12]:
import pickle
from tensorflow.keras.models import load_model

# Загружаем токенизатор
with open('tfidf_tokenizer.pkl', 'rb') as f:
    tfidf_tokenizer = pickle.load(f)

# Загружаем модель
loaded_model = load_model('text_classification_model.h5')

# Функция предсказания 
def predict_outcome(text):
    # Предобработка текста с использованием загруженного токенизатора
    text_features = tfidf_tokenizer.transform([text])
    
    # Преобразование текстовых признаков в плотный формат numpy массива
    text_features_dense = text_features.toarray()

    predictions = loaded_model.predict(text_features_dense)
    return predictions[0][0]

In [13]:
input_text = '''
Some people think that the only way to learn foreign languages is to start very early. Others believe that foreign languages can be learned at any age. In this essay I will address pluses and mines of learning foreign languages at an early age.
 On the one hand, since 4 years children talk with their parents very well. And if in family all speak on 2 languages, child will can speak on 2 languages too. It is the reason, because some people think that studying many languages is simply only at an early age. When I was younger my family often travel to different countries, when people speak in only English. And I learning to speak English to them. It was easy for me.
 On the other hand, many people move to another country. For them they must to learn another language. And they do it. That is an example of how people can learn another language at any age. I think, if people a person really wants to learn something, he can do it.
 In conclusion, somebody think that peoples can learn foreign languages at only an early age, but another think it can be done an any age. I think that everybody is right. All people can to learn foreign languages, but maybe it is easier to do it an early age.
'''


predicted_label = predict_outcome(input_text)
print("Predicted Label:", "{:.20f}".format(predicted_label))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Predicted Label: 0.99464416503906250000


# Делаем предсказание на тестовых данных

In [14]:
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
test

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [15]:
test['generated'] = test['text'].apply(predict_outcome)
submission = test[['id', 'generated']]
submission

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


,id,generated
0,0000aaaa,0.999614
1,1111bbbb,0.999614
2,2222cccc,0.999614


In [16]:
submission.to_csv('submission.csv', index=False)